In [14]:
import numpy as np
import operator
import random
import decimal as dcm
# from numpy.linalg import norm
from numpy.linalg import inv
from numpy.linalg import det
import math
from functools import reduce
import pandas as pd
# %matplotlib widget
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
plt.style.use('seaborn-whitegrid')

norm = lambda A: np.linalg.norm(A, ord=2)

### Поиск наибольшего по модулю с.ч. и соответствующего с.в

In [15]:
def get_H(n, precision):
    # функция строит матрицу Гильберта размером n x n и округляет значения до precision знаков после запятой
    a = lambda i, j: round(1 / (i + j + 1), precision)
    return np.array([[a(i, j) for j in range(n)] for i in range(n)])

def get_symmetric(n, rng=1, seed=None):
    # строит произвольную симметричную матрицу n x n, можно задать seed
    np.random.seed(seed=seed)
    A = np.random.rand(n, n) - 0.5 * np.ones((n, n))
    return (A + A.T) * rng

In [16]:
def output(method, matrix):
    # функция для вывода
    for eps in np.logspace(-2, -10, 9):
        print(f"eps: {eps:.0e}  ", end=' ')
        method(matrix, eps, False)

In [17]:
def eval_eig(x_curr, x_next):
    # оценка модуля с.ч.
    num = np.dot(x_next, x_next)
    den = np.dot(x_curr, x_curr)

    return np.sqrt(num / den)

In [18]:
def power_iter(A, eps, verbose=True):
    # Степенной метод
    n = len(A)
    x_curr = np.array([42] * n)
    x_next = A @ x_curr
    eig_curr = x_next[0] / x_curr[0]
    eig_next = x_next[0] / x_curr[0]

    iters = 0
    
    while True: 
        # нормируем вектор, если он стал слишком большим
        if eval_eig(x_curr, x_next) > 1 and norm(x_next) > 1e5:
            x_next /= norm(x_next)

        iters += 1
        x_curr, x_next = x_next, A @ x_next
        eig_curr, eig_next = eig_next, x_next[0] / x_curr[0]
        
        if abs(eig_curr - eig_next) < eps:
            break

    L = x_next[0] / x_curr[0] # искомое с.ч.
    x = x_next                # с.в.
    print("iterations:", "{:6}".format(iters))
    if verbose:
        print("Abs max eigenvalue:", L)
        print("Eigenvector:", list(x))
        np_eig = max(np.linalg.eig(matrix)[0])
        if abs(min(np.linalg.eig(matrix)[0])) > np_eig:
            np_eig = min(np.linalg.eig(matrix)[0])
        print("diff with np:", abs(L - np_eig))

***
#### Метод скалярных произведений

In [19]:
def eval_eig2(x_curr, x_next, y_next):
    # оценка с.ч.
    num = np.dot(x_next, y_next)
    den = np.dot(x_curr, y_next)

    return num / den

In [20]:
def scalar_method(A, eps, verbose=True):
    # Метод скалярных произведений
    n = len(A)
    x_curr = np.array([42] * n)
    x_next = A @ x_curr

    y_curr = np.array([37] * n)
    y_next = A.T @ y_curr

    eig_curr = eval_eig2(x_curr, x_next, y_next)
    eig_next = eval_eig2(x_curr, x_next, y_next)

    iters = 0
    
    while True: 
        if abs(eval_eig2(x_curr, x_next, y_next)) > 1 and (norm(x_next) > 1e10 or norm(y_next) > 1e10):
            x_next /= norm(x_next)
            y_next /= norm(y_next)

        iters += 1
        x_curr, x_next = x_next, A @ x_next
        y_curr, y_next = y_next, A.T @ y_next

        eig_curr, eig_next = eig_next, eval_eig2(x_curr, x_next, y_next)
        if abs(eig_curr - eig_next) < eps:
            break
        
    L = eval_eig2(x_curr, x_next, y_next) # с.ч.
    x = x_next                            # с.в.
    
    print("iterations:", "{:6}".format(iters))
    if verbose:
        print("Abs max eigenvalue:", L)
        print("Eigenvector:", list(x))
        np_eig = max(np.linalg.eig(matrix)[0])
        if abs(min(np.linalg.eig(matrix)[0])) > np_eig:
            np_eig = min(np.linalg.eig(matrix)[0])
        print("diff with np:", abs(L - np_eig))

***
#### Тесты

In [21]:
n = 100
matrix = get_H(n, 5)
eps = 1e-8

power_iter(matrix, eps)
output(power_iter, matrix)
print()
scalar_method(matrix, eps)
output(scalar_method, matrix)

iterations:     20
Abs max eigenvalue: 2.182692515645891
Eigenvector: [13606.029983648681, 9073.188166908933, 7184.6304850094575, 6079.672690909227, 5331.823705986919, 4782.4055231524435, 4356.809264410914, 4014.698175387078, 3732.061154382224, 3493.5623216965964, 3288.8821235719724, 3110.7934084562544, 2954.106079907445, 2814.8994486571282, 2690.167482177105, 2577.610891648935, 2475.437073558768, 2382.1988461606657, 2296.6056204694646, 2217.780864052889, 2144.85846717475, 2077.151589285634, 2014.1434650554556, 1955.2516397483223, 1900.0739560732068, 1848.2837680691, 1799.5582193296764, 1753.561379051716, 1710.1427288411123, 1669.0298592831905, 1630.0517568859084, 1592.9883579213758, 1557.7372194729235, 1524.17495433267, 1492.1518120127469, 1461.5703131666014, 1432.30031116164, 1404.2708652359097, 1377.3884005834238, 1351.6277237761692, 1326.8825197621836, 1303.1002587563382, 1280.2193889745133, 1258.1576701330862, 1236.89679050111, 1216.4307580958655, 1196.659840313696, 1177.537193650

In [22]:
n = 200
matrix = get_H(n, 50)
eps = 1e-14

power_iter(matrix, eps)
output(power_iter, matrix)
print()
scalar_method(matrix, eps)
output(scalar_method, matrix)

iterations:     39
Abs max eigenvalue: 2.274266987431885
Eigenvector: [5.828045185982752, 3.9467108435916334, 3.1579102000688675, 2.694104730806326, 2.3788944005948385, 2.146474184434134, 1.9658523497063882, 1.8202253681560272, 1.6995721482986519, 1.5974894684599996, 1.5096655857823755, 1.4330748266954636, 1.36552202144944, 1.3053701122495673, 1.2513697142766156, 1.2025483493731883, 1.158136109400968, 1.1175143717553941, 1.0801795599134683, 1.0457169911979398, 1.0137816497858776, 0.9840838150271092, 0.9563781582445902, 0.9304553593907245, 0.9061355824489534, 0.8832633409938435, 0.8617034166391094, 0.8413375841882369, 0.822061961460375, 0.8037848475976225, 0.7864249468311478, 0.769909898981298, 0.7541750559644045, 0.739162457048923, 0.7248199657829212, 0.7111005392774208, 0.697961606499972, 0.6853645368606777, 0.6732741839873644, 0.661658492429571, 0.6504881572821025, 0.6397363285125937, 0.6293783532152712, 0.6193915501721232, 0.6097550120420061, 0.6004494312634024, 0.5914569463829057, 

In [23]:
n = 100
matrix = get_symmetric(n, 10, seed=42)
eps = 1e-10

power_iter(matrix, eps)
output(power_iter, matrix)
print()
scalar_method(matrix, eps)
output(scalar_method, matrix)

iterations:   1175
Abs max eigenvalue: 79.12404056845186
Eigenvector: [-2.7276393114379833, 10.53036001846837, -7.496333630955807, 2.6094804271438616, 5.411807670176851, 10.877278369612776, 3.948793971096178, -5.07780368986908, 9.901764554574811, -6.1583396995446655, 9.571558008270173, 7.061991391061199, -11.21236729501219, 8.087967151067627, -6.375066967313271, -5.689836148260734, 1.204389176910967, -5.127187501299327, 11.13708982417418, -13.520942142119594, -5.567613831554173, 0.05231377308246121, 7.281788522091457, -9.221875041674242, 0.7771948697089428, -1.2454777852232137, -1.7932965472275304, 18.658049923197634, -9.095552512431938, 8.151992022544439, 4.67961775765879, -2.2597300532214613, 3.51311370442783, -0.3125367766313596, 18.313747325318595, -7.846861474438703, 10.262999878476133, -5.802960944454979, 5.949467967414553, 7.630747361770025, -5.93142448815052, 0.12441216587820847, -15.559242424866632, 11.987779267624425, 1.9213060028020943, 0.4941495533259326, 0.3146116082931651

In [24]:
n = 100
matrix = get_symmetric(n, 10, seed=451)
eps = 1e-10

power_iter(matrix, eps)
output(power_iter, matrix)
print()
scalar_method(matrix, eps)
output(scalar_method, matrix)

iterations:   3337
Abs max eigenvalue: -78.24161638096376
Eigenvector: [-51603.47845755337, -76270.94194885054, 59295.84492952419, 8341.773781751977, 2557.66267614388, 24711.147429836958, 14810.02256062376, 88092.10852942607, 9990.22832704529, 57421.55620730668, -72455.21854180076, -78077.10504533435, -5373.908515671585, 6979.977113194157, -113007.57432192023, -16257.745686619846, -62330.46383240746, 123501.81009257646, -22345.689272193864, -76698.52930708978, -30615.112904275244, -40664.97710863776, 57594.350007942005, -36507.64050378647, -16004.575710152654, -4882.0721203213425, 9497.188710778424, -56051.99670472671, 46073.780250448595, 54612.52186862973, 12855.035937833054, -40905.0515211366, 15127.789257449025, -1517.562377023216, 2829.244419740069, -74555.32616476223, 9274.810846168373, -36208.89790651006, -64918.77900471615, 43873.55864015226, 37951.61669830412, 23236.654239232776, 16168.431704718714, -71916.31453871352, 29287.496082829894, -29493.54338141348, -16780.308523023657

In [25]:
n = 100
matrix = get_symmetric(n, 100, seed=1984)
eps = 1e-10

power_iter(matrix, eps)
output(power_iter, matrix)
print()
scalar_method(matrix, eps)
output(scalar_method, matrix)

iterations:   2424
Abs max eigenvalue: -810.6259754330974
Eigenvector: [-42755.66275723254, -27668.492152055507, 123858.741856134, -17628.254043626206, -28813.986575074956, 94262.03787345429, -24067.43412983153, 17925.18973817693, -153541.0997501221, 61750.83439017584, -9699.03523865006, -22928.54486630224, 7393.759333127418, 20696.879742126206, 48787.1492955884, -51491.303673657254, -109046.83817455897, -2956.3452399468297, 21654.8154565929, -64393.675639067435, -32631.22437648855, 13221.591366833083, -80395.1811726345, 28807.98333510058, 42960.22473053973, 4399.58792789749, 94786.82145545277, -48490.72353266014, -48804.71116114628, -131869.13407097606, 85093.98950262093, 129701.58329247209, -93820.33562602085, 37334.22420764503, 96946.43256233263, -3365.3028670755193, -170499.62982720736, -106293.46829106854, 989.1313032264061, -59643.179977198684, -106913.16362637229, -62798.981864810004, -19278.506251753966, -96439.47337257756, 10642.767040258866, -57764.83354869681, 37512.27018099

In [26]:
# Для сравнения с 6 заданием
n = 50
matrix = get_symmetric(n, 10, seed=42)
eps = 1e-7

power_iter(matrix, eps)
print()
scalar_method(matrix, eps)

matrix = get_H(100, 10)
eps = 1e-10

power_iter(matrix, eps)
print()
scalar_method(matrix, eps)

iterations:    331
Abs max eigenvalue: -55.02260959598172
Eigenvector: [5760.299024599721, -6465.3222254876755, 16758.619360770972, 8311.240344073354, -55519.11305046722, -6429.745769449143, -8014.208257191497, -19320.437128247602, -5949.60289947211, -19071.69169578975, 17605.00016982763, 13996.530825601847, -24459.61811632407, 10726.825827655848, 17993.008136722478, -16790.947930948867, -5998.963518925638, 8874.963610244664, 21086.344442776288, 4310.600827546276, 36064.98750651236, -861.1108543828573, -24380.677808048298, -4036.041364103753, -40350.99328863443, 21017.333223108002, 38129.946392780854, -23466.537921509887, 26097.394665240652, -25977.901193221274, 38781.813559593305, -16893.79697136894, -27614.905783604314, 6942.192549039414, 8683.187355849655, 26800.855506079068, 11548.619094594425, -20025.139148156173, 37048.13114437623, 35803.77041143157, 10060.256601052946, 19196.387813881833, -4018.692496519513, -21270.646536667817, -18656.331250012205, 47750.11666093503, 4296.72852